# Genetic tournament to determine minimax static evaluation component weights

## Problematic

The `evaluate()` function from the `StaticEvaluation` class, used within the `minimax()` function, returns the evaluation of a board position based on six components :

- `coin_parity()`
- `actual_mobility()`
- `potential_mobility()`
- `corners_captured(game)`
- `future_corners_captured()`
- `static_weights()`

The remaining question is : **how to balance each component to produce the most effective algorithm to play Othello?**

To answer this question, we can create a **genetic tournament** where different versions of the `evaluate()` function (with different component weights) will evolve and compete against each other until we find the most effective for a given number of iterations.

## Genetic tournament

The general algorithm of a genetic algorithm is the following :

1. Randomly initialize a population of $N_{chromosomes}$ **chromosomes**
2. Repeat for $N_{generations}$ generations
    1. Determine the **fitness** of each chromosome
    2. Create an **elitism** by keeping the chromosomes with the highest fitness
    3. Randomly select parents based on their fitness
    4. Generate offspring by **crossover**
    5. Perform **mutation** on the offspring
    6. Merge the elite chromosomes and the offspring to create the next population

### Chromosome

A **chromosome** will be a list of numbers (called genes) representing the balance between all the different components.

`c = (0.1, 0.2, 0.4, 0.05, 0.25)`

The sum of all the numbers will add up to 1, as they each represent a proportion of the global evaluation.

### Fitness

The **fitness** of a chromosome will represent how strong the component's balance is at playing Othello.

This strongness will be measured by making each pair of chromosomes in a given population play against each other with a $D_{fitness}$ depth of search, once as black and once as white, and counting the ratio of win-loss.

### Elitism

To ensure the average fitness of our population is increasing with each generation, we will create an **elitism** by keeping $N_{elites}$ chromosomes with the highest fitness for the next generation.

### Crossover

A **crossover** is a method used to create offspring with parent chromosomes.

Among the plethora of crossover methods that exist, we will use **one-point**, **two-point** and **uniform** crossovers.

#### One-point crossover

**One-point crossover** creates two children from two parents' chromosomes by swapping their genes that are before a randomly picked point.

```
# Parents
p1 = (a, b, c, d, e)
p2 = (f, g, h, i, j)

# One-point crossover at index 1

# Children
c1 = (f, g, h, d, e)
c2 = (a, b, c, i, j)
```

Due to our balance constraint, we will need to normalize each  new child to ensure the sum of their genes adds up to 1.

#### Two-point crossover

**Two-point crossover** creates two children from two parents' chromosomes by swapping their genes between two distinct, randomly picked points.

```
# Parents
p1 = (a, b, c, d, e)
p2 = (f, g, h, i, j)

# One-point crossover at index 0 and index 3

# Children
c1 = (a, g, h, i, e)
c2 = (f, b, c, d, j)
```

Due to our balance constraint, we will need to normalize each  new child to ensure the sum of their genes adds up to 1.

#### Uniform crossover

**Uniform crossover** creates two children from two parents' chromosomes by randomly swapping each parent's corresponding genes with a probability $P_{swap}$.

```
# Parents
p1 = (a, b, c, d, e)
p2 = (f, g, h, i, j)

# Binary swap mask (0 -> don't swap, 1 -> swap)
bm = (0, 1, 1, 0, 1)

# Children
c1 = (a, b, c, i, j)
c2 = (f, g, g, d, e)
```

Due to our balance constraint, we will need to normalize each new child to ensure the sum of their genes adds up to 1.

### Mutation

**Mutation** is used to create diversity within our population of fresh children by tweaking, with a low $P_{mutate}$ probability, some chromosomes' genes.

As for crossover, there are plenty of methods to do it. We will focus our efforts on the **swap**, **scramble** and **inversion** mutations.

#### Swap mutation

The **swap mutation** randomly selects two distinct genes from a chromosome and swaps them.

```
# Chromosome
c1 = (a, b, c, d, e)

# Swap genes 0 and 3

# Mutation
m1 = (d, b, c, a, e)
```

#### Scramble mutation

The **scramble mutation** randomly selects a subsequence of genes from a chromosome and randomly scrambles it.

```
# Chromosome
c1 = (a, b, c, d, e)

# Subsequence 0 to 3

# Mutation
m1 = (b, d, c, a, e)
```

#### Inversion mutation

The **inversion mutation** randomly selects a subsequence of genes from a chromosome and reverses it.

```
# Chromosome
c1 = (a, b, c, d, e)

# Subsequence 0 to 3

# Mutation
m1 = (d, c, b, a, e)
```

## Implementation

### Imports

In [3]:
import sys
sys.path.append('..')

In [24]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from genetic.crossover import (
    one_point_crossover,
    two_point_crossover,
    uniform_crossover,
)
from genetic.fitness import play_tournament
from genetic.mutate import inversion, scramble, swap
from genetic.population import generate_random_population

### Parameters

In [13]:
N_CHROMOSOMES = 2
N_GENERATIONS = 3
N_ELITES = 0

# Make sure N_CHROMOSOMES - N_ELITES is divisible by 2

D_FITNESS = 1
P_SWAP = 0.5
P_MUTATE = 0.1

MAX_FITNESS = (N_CHROMOSOMES - 1) * 2
MIN_FITNESS = -MAX_FITNESS

### Random

In [9]:
np.random.seed(42)
rng = np.random.default_rng()

### Evolution

#### Metrics

In [19]:
evolution_population = []
evolution_fitnesses = []

#### Loop

In [20]:
# 1. Randomly initialize a population of chromosomes
population = generate_random_population(N_CHROMOSOMES)

# 2. Repeat for each generation
for i in tqdm(range(N_GENERATIONS)):
    # 1. Determine the fitness of each chromosome
    fitnesses = play_tournament(population, depth=D_FITNESS)

    evolution_population.append(population)
    evolution_fitnesses.append(fitnesses)

    # 2. Create an elitism by keeping the chromosomes with the highest fitness
    if N_ELITES == 0:
        elites = np.array([], dtype=float)
    else:
        # https://stackoverflow.com/a/23734295
        max_fitness_indices = np.argpartition(fitnesses, -N_ELITES)[N_ELITES:]
        elites = population[max_fitness_indices]

    # 3. Randomly select parents based on their fitness
    if fitnesses.max == fitnesses.min:
        weights = np.full(len(fitnesses), 1 / len(fitnesses)) 
    else:
        fitnesses_min_max = (fitnesses - MIN_FITNESS) / (MAX_FITNESS - MIN_FITNESS)
        weights = fitnesses_min_max / fitnesses_min_max.sum()

    parents_indices = rng.choice(N_CHROMOSOMES, size=(int((N_CHROMOSOMES - N_ELITES) / 2), 2), p=weights)
    parents = population[parents_indices]

    # 4. Generate offspring by crossover
    offsprings = np.empty(N_CHROMOSOMES - N_ELITES, dtype="object")

    for i, (p1, p2) in enumerate(parents):
        r = rng.choice([0, 1, 2], size=1)

        if r == 0:
            c1, c2 = one_point_crossover(p1, p2)
        elif r == 1:
            c1, c2 = two_point_crossover(p1, p2)
        else:
            c1, c2 = uniform_crossover(p1, p2, P_SWAP)

        offsprings[2 * i], offsprings[(2 * i )+ 1] = c1, c2

    offsprings = np.array(list(offsprings), dtype=float)  # Fix shape issue with concatenation

    # 5. Perform mutation on the offspring
    for c in offsprings:
        if rng.random() > P_MUTATE:
            continue

        r = rng.choice([0, 1, 2], size=1)

        if r == 0:
            c = swap(c)
        elif r == 1:
            c = scramble(c)
        else:
            c = inversion(c)

    # 6. Merge the elite chromosomes and the offspring to create the next population
    if N_ELITES == 0:
        next_population = offsprings
    else:
        next_population = np.concatenate((elites, offsprings))

    population = next_population.copy()

evolution_population = np.array(evolution_population)
evolution_fitnesses = np.array(evolution_fitnesses)

100%|██████████| 3/3 [12:04<00:00, 241.58s/it]


### Save results

#### Locations

In [27]:
population_path = '../data/genetic_population_evolution.csv'
fitnesses_path = '../data/genetic_fitnesses_evolution.csv'

#### Save to csv

In [100]:
columns = pd.MultiIndex.from_product([['score'], [f"chromosome_{i}" for i in range(N_CHROMOSOMES)]])

df_fitnesses = pd.DataFrame(evolution_fitnesses, columns=columns)
df_fitnesses.index.name = "generation"

df_fitnesses.head()

score             
           chromosome_0 chromosome_1
generation                          
0                    -2            2
1                     0            0
2                     0            0

In [101]:
reshaped_data = evolution_population.reshape(-1, evolution_population.shape[-1])

index = np.repeat(np.arange(evolution_population.shape[0]), evolution_population.shape[1])
columns=["coin_parity",
         "actual_mobility",
         "potential_mobility",
         "corners_captured",
         "future_corners_captured",
         "static_weights"]

df_population = pd.DataFrame(reshaped_data, columns=columns)
df_population['generation'] = index
df_population.set_index('generation', inplace=True)

df_population.head()

,coin_parity,actual_mobility,potential_mobility,corners_captured,future_corners_captured,static_weights
generation,,,,,,
0,0.264118,0.130153,0.290358,0.045364,0.222582,0.047424
0,0.194706,0.174473,0.051695,0.202613,0.193768,0.182747
1,0.194706,0.174473,0.051695,0.202613,0.193768,0.182747
1,0.194706,0.174473,0.051695,0.202613,0.193768,0.182747
2,0.194706,0.174473,0.051695,0.202613,0.193768,0.182747


In [102]:
df_population.to_csv(population_path)
df_fitnesses.to_csv(fitnesses_path)